In [2]:
library(Seurat)
library(Matrix)
library(reticulate)
library(glmpca)
library(peakRAM)
library(scry)
library(SingleCellExperiment)
library(glmGamPoi)
library(harmony)

# Load data

In [3]:
mtx <- readMM('/data01/hanbin973/fastrna_paper/datasets/pbmc_ssc/mat.mtx')
meta <- read.csv('/data01/hanbin973/fastrna_paper/datasets/pbmc_ssc/meta.csv', row.names=1)
colnames(mtx) <- rownames(meta)
rownames(mtx) <- 1:nrow(mtx)
covmat <- model.matrix(as.formula('~factor(Method)'), meta)
covmat <- covmat[,2:ncol(covmat)]
meta[,paste0('batch_',1:ncol(covmat))] <- covmat

# Seurat pipeline - log

In [3]:
benchmark_seurat_log <- function(obj){
    obj <- NormalizeData(obj, verbose=FALSE)
    
    obj.list <- SplitObject(obj, split.by="Method")
    obj.list <- lapply(X=obj.list, FUN=function(x){
        x <- NormalizeData(x, verbose=FALSE)
        x <- FindVariableFeatures(x, verbose=FALSE)
    })
    features <- SelectIntegrationFeatures(object.list=obj.list, verbose=FALSE)
    obj.list <- lapply(X=obj.list, FUN=function(x){
        x <- ScaleData(x, features=features, verbose=FALSE)
        x <- RunPCA(x, features=features, verbose=FALSE)
    })
    
    obj.anchors <- FindIntegrationAnchors(object.list=obj.list, anchor.features=features, reduction='rpca', verbose=FALSE)
    obj.combined <- IntegrateData(anchorset=obj.anchors, verbose=FALSE)
    DefaultAssay(obj.combined) <- "integrated"
    obj.combined <- ScaleData(obj.combined, verbose=FALSE)
    obj.combined <- RunPCA(obj.combined, npcs=20, verbose=FALSE)
    
    write.csv(Embeddings(obj.combined), 'pca_coord/pbmcssc/pca_log.csv')
    obj.combined
}

In [4]:
obj <- CreateSeuratObject(mtx, meta.data=meta)
result <- peakRAM(x <- benchmark_seurat_log(obj))

In [5]:
result[,c('Elapsed_Time_sec', 'Peak_RAM_Used_MiB')]

,Elapsed_Time_sec,Peak_RAM_Used_MiB
,<dbl>,<dbl>
1,81.031,2964


# Seurat pipeline - sct

In [6]:
benchmark_seurat_sct<- function(obj){
    obj <- SCTransform(obj, variable.features.n=2000, 
                       latent_var=c('log_umi', paste0('batch_',1:ncol(covmat))), 
                       method="glmGamPoi",
                      verbose=FALSE)
    obj <- RunPCA(obj, verbose=FALSE)
    write.csv(Embeddings(obj), 'pca_coord/pbmcssc/pca_sct.csv')
    obj
}

In [7]:
obj <- CreateSeuratObject(mtx, meta.data=meta)
result <- peakRAM(x <- benchmark_seurat_sct(obj))

In [8]:
result[,c('Elapsed_Time_sec', 'Peak_RAM_Used_MiB')]

,Elapsed_Time_sec,Peak_RAM_Used_MiB
,<dbl>,<dbl>
1,194.176,7163.9


# glmpca

In [9]:
benchmark_scry_glmpca <- function(obj){
    obj <- devianceFeatureSelection(obj, assay="counts", sorted=TRUE)
    obj2 <- obj[1:2000,]
    obj2 <- GLMPCA(obj2, 50, assay="counts", minibatch="stochastic", X=as.matrix(colData(obj2)))
    write.csv(metadata(obj2)$glmpca$factors, 'pca_coord/pbmcssc/pca_glmpca.csv')
}

In [10]:
obj <- SingleCellExperiment(list(counts=mtx))
colData(obj)[,paste0('batch_',1:ncol(covmat))] <- covmat
result <- peakRAM(x <- benchmark_scry_glmpca(obj))

In [12]:
result[,c('Elapsed_Time_sec', 'Peak_RAM_Used_MiB')]

,Elapsed_Time_sec,Peak_RAM_Used_MiB
,<dbl>,<dbl>
1,1121.989,3383.4


# Harmony

In [8]:
benchmark_harmony <- function(obj){
    obj <- NormalizeData(obj)
    obj <- FindVariableFeatures(obj, verbose=FALSE, nfeatures=2000)
    obj <- ScaleData(obj, verbose=FALSE)
    obj <- RunPCA(obj, verbose=FALSE)
    obj <- obj %>% 
        RunHarmony("Method", plot_convergence = FALSE, verbose=FALSE)
    
    write.csv(Embeddings(obj, 'harmony'), 'pca_coord/pbmcssc/pca_harmony.csv')
    obj
}

In [9]:
obj <- CreateSeuratObject(mtx, meta.data=meta)
result <- peakRAM(x <- benchmark_harmony(obj))

Warning message:
“Invalid name supplied, making object name syntactically valid. New object name is Seurat..ProjectDim.RNA.harmony; see ?make.names for more details on syntax validity”


In [10]:
result

,Function_Call,Elapsed_Time_sec,Total_RAM_Used_MiB,Peak_RAM_Used_MiB
,<chr>,<dbl>,<dbl>,<dbl>
1,x<-benchmark_harmony(obj),37.08,-106.1,758
